# **Validation per Label Class**

In [70]:
import pandas as pd
import zipfile
import json

In [14]:
subreddits =  ["Ask_Politics", "Askpolitics", "NeutralPolitics", "PoliticalDiscussion", "PoliticalDebate", "politics"]

In [2]:

Ask_Politics_labels = pd.read_csv("output_reddit/Ask_Politics_Llama_3.1_8B_ft_classified.csv")
Askpolitics_labels = pd.read_csv("output_reddit/Askpolitics_Llama_3.1_8B_ft_classified.csv")

PoliticalDebate_labels = pd.read_csv("output_reddit/PoliticalDebate_Llama_3.1_8B_ft_classified.csv")
PoliticalDiscussion_labels = pd.read_csv("output_reddit/PoliticalDiscussion_Llama_3.1_8B_ft_classified.csv")

NeutralPolitics_labels = pd.read_csv("output_reddit/NeutralPolitics_Llama_3.1_8B_ft_classified.csv")
 


labels = {
    'Ask_Politics' : Ask_Politics_labels,
    'Askpolitics' : Askpolitics_labels,
    'PoliticalDiscussion' : PoliticalDiscussion_labels,
    'PoliticalDebate' : PoliticalDebate_labels,
    'NeutralPolitics' : NeutralPolitics_labels,
    #'politics' : politics_labels
    
}

In [3]:
politics_labels = pd.DataFrame()

for i in range(6):
    
    pol_part = pd.read_csv(f"output_reddit/politics_{i}_Llama_3.1_8B_ft_classified.csv")
    politics_labels = pd.concat([politics_labels, pol_part]).reset_index(drop = True)

labels['politics'] = politics_labels

In [4]:
labels['politics']

,msg_id_parent,msg_id_child,submission_id,predictions_label_ft,predictions_score_ft,predictions_ft
0,c0ae9r5,c0aeb6y,8tp50,0,0.8335,no_disagreement
1,c0aeds9,c0aee4d,8tp50,0,0.9727,no_disagreement
2,c0aeds9,c0aeglj,8tp50,1,0.8610,disagree
3,c0aeds9,c0aehqw,8tp50,1,0.9966,disagree
4,c0aecro,c0aegcb,8tp50,1,0.9920,disagree
...,...,...,...,...,...,...
4168047,kfkh5ib,kfmd52k,18uirzt,0,0.9585,no_disagreement
4168048,kfklsb6,kforzbx,18uirzt,0,0.9950,no_disagreement
4168049,kfmq5kr,kfp6tzv,18uirzt,1,0.7095,disagree
4168050,kfp3d4v,kfp8xh8,18uirzt,0,0.9870,no_disagreement


### **Get sample of 10 per label and per subreddit, each**

In [15]:
val_sample = {}


for subr in subreddits:

    df_temp = labels[subr]
    val_sample[subr] = df_temp.groupby('predictions_ft')[['predictions_ft', 'msg_id_parent', 'msg_id_child', 'submission_id']].sample(n = 10, random_state = 23)




In [16]:
val_sample

{'Ask_Politics':         predictions_ft msg_id_parent msg_id_child submission_id
 42310         disagree       fy1hc7f      fy2fgbe        hr2p40
 26373         disagree       e05bdj1      e05bjg7        8opcg1
 37972         disagree       fjc4p1q      fjcpxs3        fckbtt
 25560         disagree       dwbk2u3      dwbkimy        87aqoi
 38695         disagree       fl6a1af      fliv3y0        fm4lzh
 26607         disagree       e10h6ws      e10w7tn        8sle0h
 32682         disagree       es9r670      esacnwh        c6mfke
 26912         disagree       e274pi2      e278msy        8xsz3c
 45730         disagree       gc833q1      gc8ulgn        jtu90n
 17633         disagree       d8drwta      d8dw7xg        55teif
 16364  no_disagreement       d5g2qw3      d5g37bc        4ta8us
 18318  no_disagreement       d9s6gqi      d9s7zxl        5by0xf
 46199  no_disagreement       geapmlg      geayfcf        k4t55z
 10739  no_disagreement       ctrkz4v      ctrl3sx        3frxiy
 21877  n

### **Get texts**

In [17]:
# reddit data
Ask_Pol_path = "../../../../data/Ask_Politics_new_preprocessing_com_rep.csv"
Askpol_path = "../../../../data/Askpolitics_new_preprocessing_com_rep.csv"
PolDis_path = "../../../../data/PoliticalDiscussion_new_preprocessing_com_rep.csv"
PolDeb_path = "../../../../data/PoliticalDebate_new_preprocessing_com_rep.csv"
NeuPol_path = "../../../../data/NeutralPolitics_new_preprocessing_com_rep.csv"

pol_path = "../../../../politics_preprocess/output/splitted_politics_data.zip"


In [18]:
# import data
Ask_Politics = pd.read_csv(Ask_Pol_path)
Askpolitics = pd.read_csv(Askpol_path)
PoliticalDiscussion = pd.read_csv(PolDis_path)
PoliticalDebate = pd.read_csv(PolDeb_path)
NeutralPolitics = pd.read_csv(NeuPol_path)


In [21]:

# politics from zip files
df_chunks = []
with zipfile.ZipFile(pol_path, 'r') as zf:
    file_list = zf.namelist()  # Get all file names in ZIP

    # Sort to maintain order (if needed)
    file_list.sort()  

    print("Files found in ZIP:", file_list)  # Debugging step

    for file in file_list:
        with zf.open(file) as f:
            df_chunk = pd.read_csv(f,  quotechar='"', encoding="utf-8")  
            df_chunks.append(df_chunk)  # Store the chunk


politics = pd.concat(df_chunks, ignore_index=True)

print(f"Successfully loaded {len(politics)} rows from {pol_path}")

Files found in ZIP: ['data_part_1.csv', 'data_part_2.csv', 'data_part_3.csv', 'data_part_4.csv', 'data_part_5.csv']
Successfully loaded 4175294 rows from ../../../../politics_preprocess/output/splitted_politics_data.zip


In [61]:
text_data = {
    'Ask_Politics' : Ask_Politics,
    'Askpolitics' : Askpolitics,
    'PoliticalDiscussion' : PoliticalDiscussion,
    'PoliticalDebate' : PoliticalDebate,
    'NeutralPolitics' : NeutralPolitics,
    'politics' : politics
}

In [62]:
for subr in ["politics", "PoliticalDiscussion"]:
    for col in ['msg_id_parent', 'msg_id_child', 'submission_id',
                'subreddit', 
                'body_child', 'body_parent', 'submission_text',
                'author_child', 'author_parent', 'author_submission']:
        if text_data[subr][col].dtype == "object":  # Apply only to string columns
            text_data[subr][col] = text_data[subr][col].str.strip('"')


In [64]:
text_data['PoliticalDiscussion']

,msg_id_parent,msg_id_child,body_child,subreddit,exact_time_child,author_child,ups_child,score_child,downs_child,submission_id,...,author_parent,ups_parent,score_parent,downs_parent,exact_time_parent,submission_text,author_submission,ups_submission,score_submission,exact_time_submission
0,c1pvucb,c1pwq17,Well thanks for having the patience to wade th...,PoliticalDiscussion,1.303074e+09,dtmike07,2.0,2,0.0,grqj9,...,theiamina,0.0,0,0.0,1.303063e+09,I've been an adherent of or flirted with most ...,dtmike07,15.0,7,1.302996e+09
1,c1pw8hh,c1pwsh4,Very good point - I felt bad using the labels ...,PoliticalDiscussion,1.303075e+09,dtmike07,0.0,0,0.0,grqj9,...,avd007,0.0,0,0.0,1.303068e+09,I've been an adherent of or flirted with most ...,dtmike07,15.0,7,1.302996e+09
2,c1pwkxk,c1pyoun,Well that was insightful. Thanks Reminds me of...,PoliticalDiscussion,1.303098e+09,snorky94,0.0,0,0.0,grsi7,...,ReferentiallySeethru,2.0,2,0.0,1.303072e+09,"Oops, I mean bipartisanship.",snorky94,10.0,3,1.303003e+09
3,c1rzgqn,c1rzi35,Karma Circlejerks of this magnitude are few an...,PoliticalDiscussion,1.304315e+09,funkinthetrunk,5.0,5,0.0,h1vss,...,daychilde,3.0,3,0.0,1.304315e+09,Does it make Obama look tough on terror? Will ...,funkinthetrunk,11.0,10,1.304308e+09
4,c1s5jhd,c1s5lgd,I didn't know there was a withdrawal scheduled...,PoliticalDiscussion,1.304397e+09,funkinthetrunk,1.0,1,0.0,h1vss,...,ACHILLESiii,2.0,2,0.0,1.304396e+09,Does it make Obama look tough on terror? Will ...,funkinthetrunk,11.0,10,1.304308e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668128,kfqdbma,kfrien2,I meant the legislature members from a given p...,PoliticalDiscussion,1.704064e+09,Awesomeuser90,2.0,2,0.0,18vas2y,...,Sands43,16.0,16,0.0,1.704049e+09,America is reputed for their legislature with ...,Awesomeuser90,16.0,16,1.704039e+09
668129,kfpr5h6,kfrj1xb,I was more so alluding to the reputation of th...,PoliticalDiscussion,1.704065e+09,Awesomeuser90,2.0,2,0.0,18vas2y,...,CorporateNonperson,18.0,18,0.0,1.704041e+09,America is reputed for their legislature with ...,Awesomeuser90,16.0,16,1.704039e+09
668130,kfn0ce4,kfrl1pz,Putin has been threatening to use them for ove...,PoliticalDiscussion,1.704066e+09,gaxxzz,1.0,1,0.0,18uou00,...,BitterFuture,120.0,120,0.0,1.703987e+09,Much has been made about Putin using a nuke to...,Mad_Prog_1,35.0,35,1.703968e+09
668131,kfoiclq,kfrlbx9,She wasnt even the best black woman for the jo...,PoliticalDiscussion,1.704066e+09,kimanf,1.0,1,0.0,18u0jvx,...,SeekSeekScan,2.0,2,0.0,1.704016e+09,This assumes that in the next couple of months...,DenseYear2713,2.0,2,1.703892e+09


In [72]:
val_data_final = {}

for subr in subreddits:
    val_data_final[subr] = text_data[subr][text_data[subr]['msg_id_child'].isin(val_sample[subr]['msg_id_child'])]
    val_data_final[subr] = val_data_final[subr][['submission_id', 'msg_id_parent', 'msg_id_child', 'body_child', 'body_parent', 'submission_text', 'subreddit']]

In [73]:

val_data = pd.concat([val_data_final['Ask_Politics'], val_data_final['Askpolitics'], val_data_final['PoliticalDiscussion'], 
           val_data_final['PoliticalDebate'], val_data_final['NeutralPolitics'], val_data_final['politics']])

In [75]:
val_data.to_csv("output_class_validation/val_data.csv", index = False)

In [74]:
val_data

,submission_id,msg_id_parent,msg_id_child,body_child,body_parent,submission_text,subreddit
10933,3frxiy,ctrkz4v,ctrl3sx,I wonder how often people actually do like the...,"If I like my representative, I should be able ...",I started thinking about this after hearing so...,Ask_Politics
16660,4ta8us,d5g2qw3,d5g37bc,"Yes, good point - also, see the comment by Th...","I think you mean collective bargaining rights,...",It seems to me that unions are the only way to...,Ask_Politics
17937,55teif,d8drwta,d8dw7xg,Benghazi and Libya's destabilization are two s...,There is about 4 years of Benghazi Controversy...,"Hoping for pure facts, some debunking, some re...",Ask_Politics
18622,5by0xf,d9s6gqi,d9s7zxl,"Partisan preferences aside, I'm outright stunn...",Go take a look at the polls. He's doing much b...,He suggested multiple times that the polls are...,Ask_Politics
22216,6don14,di47c5d,di4fjwf,Bernie supporter here - it's super absurd. Als...,Russia did deny it. The conspiracy theory has ...,I've read tons of posts and articles about the...,Ask_Politics
...,...,...,...,...,...,...,...
3164336,joos6k,gb9aenj,gb9ahnr,The website has said that since Monday,Per NY Times about Pennsylvania The remaining...,Good afternoon ! Results can be found below. ...,politics
3227265,jovp08,gbapttb,gbaq1pz,It's nice to be wanted at least.,"Y'all, AZ be scaring me. I know we don't need ...",Good evening ! Results can be found below. Na...,politics
3486998,kj5k4g,gguwl9e,ggvuqt6,No president should be allowed to pardon anyone.,No president should ever be allowed to pardon ...,President Trump granted full pardons to 26 mor...,politics
3640795,ks9iz4,gifgyl4,gifiluf,They're not downvotes. The message is hidden a...,So many downvotes in this thread for calling a...,Congress has certified the results of the 2020...,politics
